In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
PATH = lambda x : os.path.join("/home/sadjad/temp/static/", x)

worker_data = pd.read_csv(PATH('workers.csv'))
summary_data = pd.read_csv(PATH('summary.csv'))

In [ ]:
d = worker_data.sort_values('workerId').groupby('workerId').agg({
    'workerId': 'max',
    'raysEnqueued': 'sum',
    'raysDequeued': 'sum',
    'bytesEnqueued': 'sum',
    'bytesDequeued': 'sum',
    'numSamples': 'sum',
    'bytesSamples': 'sum'
}).reset_index(drop=True)

data = pd.merge(d, summary_data, how='left', on='workerId', validate='1:1')

BANDWIDTH = 50_000_000 # 50 MB/s
SHADE_CUTOFF = 100
MEAN_SHADE_TIME = np.mean(data[data.shade > SHADE_CUTOFF].processTime / data[data.shade > SHADE_CUTOFF].shade)

def get_compute(d):
    if d.trace > 0:
        return d.processTime / 2
    elif d.shade > SHADE_CUTOFF:
        return d.processTime / 1.5
    elif d.shade > 0:
        return (MEAN_SHADE_TIME * d.shade) / 1.5
    else:
        return d.processTime

data['transfer'] = (data.bytesDequeued + data.bytesEnqueued + data.bytesSamples) / BANDWIDTH
data['compute'] = data.apply(get_compute, axis=1) / 1e9
data['weight'] = np.maximum(data.transfer, data.compute)

t0_weight = data[data.treeletId == 0].weight.values[0]
envmap_filter = (data.trace == 0) & (data.shade == 0) & (data.process > 0)
data.loc[envmap_filter, 'weight'] = t0_weight


u_shade = data[data.shade > 0]
u_trace = data[data.trace > 0]
u_other = data[(data.trace == 0) & (data.shade == 0)]
plt.scatter(u_shade.raysDequeued, u_shade.weight, marker='*', label='Shade')
plt.scatter(u_trace.raysDequeued, u_trace.weight, marker='+', label='Trace')
plt.scatter(u_other.raysDequeued, u_other.weight, marker='+', label='Other')
plt.xscale('log')
plt.legend()

data[data.treeletId.isin([2,3])]